<a href="https://colab.research.google.com/github/vnavya2004/BTP/blob/main/Final10_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=1993f15f5541cd6411f1e4036ce8996d579be618cfa6b375fcd3b03dea6d9f5c
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [ ]:
import torch
import pandas as pd
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification
import random
from google.colab import files

# Load the XLM-RoBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Assuming you're using Google Colab and uploaded a file
uploaded = files.upload()

# Read the Excel file
df = pd.read_excel(pd.ExcelFile(list(uploaded.keys())[0]), header=0)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Saving newenglish.xlsx to newenglish.xlsx


In [ ]:

# Separate the dataset into two based on label 0 and label 1
df_0 = df[df['labels'] == 0]
df_1 = df[df['labels'] == 1]

# Find the minimum count between both labels
min_count = min(len(df_0), len(df_1))

print(min_count)
# Reduce both datasets to the minimum count
df_0_reduced = df_0.sample(n=min_count, random_state=42)
df_1_reduced = df_1.sample(n=min_count, random_state=42)

# Concatenate both reduced datasets
df_balanced = pd.concat([df_0_reduced, df_1_reduced])

# Shuffle the dataset
df = df_balanced.sample(frac=0.2, random_state=42).reset_index(drop=True)
print(df[df['labels'] == 0])
print(df[df['labels'] == 1])

21953
                                                 tweets  labels
2                  bthats with world goingnnnnnnnnnnnnn       0
6                   bdepression welcome back sweetheart       0
8          goal good even better great some days rxexxa       0
9     pleased announce that antisocial started farm ...       0
11    quick make happy again canxexxt have depressed...       0
...                                                 ...     ...
8769  canxexxt even kids their socks laundry basket ...       0
8771  catboy maguros opinion home cant draw sorry ve...       0
8773                         cautiously optimistic plan       0
8775  didnxexxt know were susd glad backk xfxfxxaaxf...       0
8778  wasnt about brexitnxexxci work every single br...       0

[4384 rows x 2 columns]
                                                 tweets  labels
0     bpeople start using substances many reasons so...       1
1             bthe depression adds little pizazz flavor       1
3     nex

In [ ]:

tweets_column = 'tweets'
labels_column = 'labels'
NUM_LABELS = len(df[labels_column].unique())
possible_labels = df[labels_column].unique()
label_dict = {possible_label: index for index, possible_label in enumerate(possible_labels)}
df['labels'] = df[labels_column].map(label_dict)



import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

# Initialize NLTK components
nltk.download('stopwords')
nltk.download('punkt')  # Tokenizer
nltk.download('wordnet')  # Lemmatizer if needed

# Load English stopwords
stop_words = set(stopwords.words('english'))

print(stop_words)

# Preprocessing function for English text
def preprocess_english_tweet(tweet):
    tweet = str(tweet)

    # Remove URLs
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)

    # Remove special characters and digits (keeping only English letters)
    tweet = re.sub(r'[^a-zA-Z\s]', '', tweet)

    # Tokenize the tweet
    words = tweet.split()

    # Remove stopwords
    words = [word for word in words if word.lower() not in stop_words]

    # You can apply stemming or lemmatization here if desired
    words = [word for word in words]

    # Join the cleaned words back into a string
    tweet = ' '.join(words)

    return tweet

# Apply preprocessing to the 'tweets' column
df[tweets_column] = df[tweets_column].astype(str).apply(preprocess_english_tweet)

# Split dataset into labeled (20%), unlabeled (60%), and test (20%) sets
df_labeled, df_temp = train_test_split(df, stratify=df[labels_column], test_size=0.8)
df_unlabeled, df_temp_val_test = train_test_split(df_temp, stratify=df_temp[labels_column], test_size=0.25)
df_val, df_test = train_test_split(df_temp_val_test, stratify=df_temp_val_test[labels_column], test_size=0.5)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


{'herself', "mustn't", 'he', 'each', 'didn', 'yourselves', 'my', 'below', 'here', 'couldn', 'him', 'were', "needn't", 'ourselves', 'very', "you're", 'up', 'i', 'itself', "couldn't", 'will', 'in', 'at', 'again', "don't", "haven't", 'has', 'between', 'further', 'is', 'any', "mightn't", 'and', 'themselves', 'wouldn', "hadn't", 'should', 'out', 'this', 'when', 'doesn', 'myself', 'most', 'once', "hasn't", 'but', 'don', 'isn', 'hers', 'hadn', 'do', "you'll", 'ours', "didn't", 'had', 'about', 'before', 'so', 'our', "shan't", 'on', "she's", "it's", 'down', "wasn't", "you'd", 'over', 'not', 'weren', 'am', 'them', 'being', 'be', 'shouldn', "isn't", 'you', 'which', 'doing', 'with', 'ma', 'against', 'have', 'hasn', 'needn', 'they', 'she', 'under', 'own', 'what', 'yourself', 'an', 'or', 'll', 'theirs', 'there', 'through', 'only', 'too', 'who', 'haven', 'more', 'if', 'wasn', 'whom', 'because', 'aren', 'can', 'from', 'off', 'above', 'his', 'no', 'few', 'the', 'their', "aren't", 'yours', 'are', 'himse

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
df_labeled

,tweets,labels
173,louis appleby effect covid suicide ratesnnan i...,0
6016,bthat ivanovic goal depressed days,0
441,suicides deaths business death jobs teens comm...,0
8462,lawyers particularly susceptible stress anxiet...,0
5210,glad type isnxexxt drug dealers road,1
...,...,...
7370,cant understand english given range undoc data...,0
3326,bthrow back thursday nwhen depress shit lookin...,0
3077,glad youxexxre back xfxfxaxb,1
8190,bvisto nasa nasa depress astronauts spacefood ...,0


In [ ]:
df_unlabeled

,tweets,labels
4418,want happy hopeful also slightly drunk drinkin...,1
6097,heidi harts piece considers problems privilege...,0
6057,bone huge driving force behind phenomenon buye...,1
279,depression anxiety itxexxs called burn fcking ...,0
6584,ixexxm glad well xfxfxxc,1
...,...,...
6112,cautiously optimistic oscars steven soderbergh...,1
6626,glad ricky place,1
901,concern another countries affairsnyour country...,1
3825,tears enemies,0


In [ ]:
# Convert the tweets and labels columns to the desired formats
df_labeled[tweets_column] = df_labeled[tweets_column].astype(str)
df_labeled[labels_column] = df_labeled[labels_column].map(label_dict).to_numpy()

df_val[tweets_column] = df_val[tweets_column].astype(str)
df_val[labels_column] = df_val[labels_column].map(label_dict).to_numpy()
df_unlabeled[tweets_column] = df_unlabeled[tweets_column].astype(str)
# Unlabeled data has no labels, so no conversion for y_unlabeled
df_test[tweets_column] = df_test[tweets_column].astype(str)
df_test[labels_column] = df_test[labels_column].map(label_dict).to_numpy()

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Load translation models
src_to_tgt_model_name = 'Helsinki-NLP/opus-mt-en-fr'  # English to French
tgt_to_src_model_name = 'Helsinki-NLP/opus-mt-fr-en'  # French to English

src_to_tgt_model = MarianMTModel.from_pretrained(src_to_tgt_model_name)
src_to_tgt_tokenizer = MarianTokenizer.from_pretrained(src_to_tgt_model_name)

tgt_to_src_model = MarianMTModel.from_pretrained(tgt_to_src_model_name)
tgt_to_src_tokenizer = MarianTokenizer.from_pretrained(tgt_to_src_model_name)

def back_translate(texts):
    # Translate Bangla to English
    translated = src_to_tgt_tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    translated_texts = src_to_tgt_model.generate(**translated)
    translated_texts = [src_to_tgt_tokenizer.decode(t, skip_special_tokens=True) for t in translated_texts]

    # Translate English back to Bangla
    back_translated = tgt_to_src_tokenizer(translated_texts, return_tensors="pt", padding=True, truncation=True)
    back_translated_texts = tgt_to_src_model.generate(**back_translated)
    back_translated_texts = [tgt_to_src_tokenizer.decode(t, skip_special_tokens=True) for t in back_translated_texts]

    return back_translated_texts
# Generate back-translated samples for labeled data
def augment_data_with_back_translation(df_labeled, augmentation_factor=1):
    augmented_samples = []

    for _ in range(augmentation_factor):
        # Back-translate the tweets
        back_translated_texts = back_translate(df_labeled[tweets_column].tolist())
        augmented_samples.extend(back_translated_texts)

    # Create a DataFrame for augmented data
    augmented_labels = df_labeled['labels'].values.tolist() * augmentation_factor  # Same labels for augmented data
    augmented_df = pd.DataFrame({tweets_column: augmented_samples, 'labels': augmented_labels})

    return augmented_df


# Assuming df_labeled is your original DataFrame with a column named 'labels'
# Calculate the number of samples to augment for each label (20% of total)
total_samples = len(df_labeled)
num_samples_to_augment = int(total_samples * 0.2)

# Calculate samples for each label (equal number)
num_samples_per_label = num_samples_to_augment // 2

# Separate the DataFrame into two subsets: one for each label
df_label_0 = df_labeled[df_labeled['labels'] == 0]
df_label_1 = df_labeled[df_labeled['labels'] == 1]

# Randomly sample from each subset, ensuring equal representation
df_label_0_sample = df_label_0.sample(n=num_samples_per_label, random_state=42)
df_label_1_sample = df_label_1.sample(n=num_samples_per_label, random_state=42)

# Combine the sampled data
df_labeled_subset = pd.concat([df_label_0_sample, df_label_1_sample])

# Print the selected subset (for verification)
print("Subset of Labeled Data for Augmentation:")
print(df_labeled_subset)

# Augment the selected subset
augmented_data = augment_data_with_back_translation(df_labeled_subset, augmentation_factor=1)

# Display the augmented data for further use
print("Augmented Data:")
print(augmented_data)



/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Subset of Labeled Data for Augmentation:
                                                 tweets  labels
1247  optimistic looking xfxfxxabnwherever whatever ...       0
1223  glad good timemore years gods abundant blessin...       0
3915  xexxcsome cheerful news change americaxexxs ba...       0
5660  rose songs really love relate toothe song give...       0
5563                 yeah wouldve waited days cool glad       0
...                                                 ...     ...
6438  donxexxt think wexexxre depressed enough post ...       1
4668  bshes emotionally unstable depressed maybe strong       1
2966                         suicide squad james gunnnn       1
1441  heres vaccine badge displaying passport carryi...       1
1663                             depression really real       1

[86 rows x 2 columns]
Augmented Data:
                                               tweets  labels
0   optimistic search xxfxxabn where time will be ...       0
1   happy good times more ye

In [ ]:
import random
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Define random insertion, deletion, and swap functions
bangla_stopwords= set(stopwords.words('english'))

# Random insertion function for Bangla text
def random_insertion(text, n=1):
    words = text.split()  # Tokenize the Bangla sentence into words
    for _ in range(n):
        # Choose a random Bangla stopword and insert it into a random position
        new_word = random.choice(list(bangla_stopwords))
        pos = random.randint(0, len(words))
        words.insert(pos, new_word)
    return ' '.join(words)  # Join words back into a sentence

def random_deletion(text, p=0.1):
    words = text.split()
    if len(words) == 1:
        return text
    # Randomly delete words with probability `p`
    words = [word for word in words if random.random() > p]
    return ' '.join(words) if words else text

def random_swap(text, n=1):
    words = text.split()
    if len(words) < 2:
        return text
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return ' '.join(words)

# Function to apply augmentation to a subset of the unlabeled data
def augment_unlabeled_data(df_unlabeled, tweets_column, augment_type='insertion', fraction=0.1, n=1):
    num_samples_to_augment = int(fraction * len(df_unlabeled))
    df_sample = df_unlabeled.sample(n=num_samples_to_augment, random_state=42)

    augmented_texts = []
    for tweet in df_sample[tweets_column]:
        if augment_type == 'insertion':
            augmented_texts.append(random_insertion(tweet, n=n))
        elif augment_type == 'deletion':
            augmented_texts.append(random_deletion(tweet))
        elif augment_type == 'swap':
            augmented_texts.append(random_swap(tweet, n=n))

    df_sample_augmented = df_sample.copy()
    df_sample_augmented[tweets_column] = augmented_texts
    return df_sample_augmented

# Get the subsets of the unlabeled data for augmentation
# 10% for each augmentation method
df_unlabeled_insertion = augment_unlabeled_data(df_unlabeled, tweets_column, augment_type='insertion', fraction=0.06)
df_unlabeled_deletion = augment_unlabeled_data(df_unlabeled, tweets_column, augment_type='deletion', fraction=0.05)
df_unlabeled_swap = augment_unlabeled_data(df_unlabeled, tweets_column, augment_type='swap', fraction=0.06)

# Combine the augmented samples
df_unlabeled_augmented = pd.concat([df_unlabeled_insertion, df_unlabeled_deletion, df_unlabeled_swap])

# Combine the augmented data with the remaining 70% of non-augmented unlabeled data
df_unlabeled_combined = pd.concat([df_unlabeled_augmented, df_unlabeled], ignore_index=True)


print(len(df_unlabeled_combined))

6163


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

df_combined = pd.concat([df_labeled, augmented_data], ignore_index=True)

# Re-tokenize the combined dataset for training
encoded_data_train = tokenizer.batch_encode_plus(
    df_combined[tweets_column].tolist(),
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)
# Tokenize labeled data for training


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df_combined['labels'].values)

# Tokenize unlabeled data
encoded_data_unlabeled = tokenizer.batch_encode_plus(
    df_unlabeled_combined[tweets_column].tolist(),
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_unlabeled = encoded_data_unlabeled['input_ids']
attention_masks_unlabeled = encoded_data_unlabeled['attention_mask']
#tokenize validation data
encoded_data_val = tokenizer.batch_encode_plus(
    df_val[tweets_column].tolist(),
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df_val['labels'].values)

# Tokenize test data
encoded_data_test = tokenizer.batch_encode_plus(
    df_test[tweets_column].tolist(),
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df_test['labels'].values)

# Create datasets
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_unlabeled = TensorDataset(input_ids_unlabeled, attention_masks_unlabeled)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

# Custom teacher model with dropout
class CustomTeacherModel(torch.nn.Module):
    def __init__(self, dropout_rate, num_labels):
        super(CustomTeacherModel, self).__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=num_labels)
        self.dropout = torch.nn.Dropout(dropout_rate)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        logits = self.dropout(outputs.logits)
        return outputs

# Function to initialize teacher models with different seeds
def initialize_teacher_model(seed, dropout_rate):
    torch.manual_seed(seed)
    return CustomTeacherModel(dropout_rate, NUM_LABELS)

# Define seeds and dropout rates for each teacher model
seeds = [42, 43, 44]
dropout_rates = [0.1, 0.2, 0.3]

# Initialize teacher models
# Initialize student and teacher models
student_model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=NUM_LABELS)
teacher_model1 = initialize_teacher_model(seeds[0], dropout_rates[0])
teacher_model2 = initialize_teacher_model(seeds[1], dropout_rates[1])
teacher_model3 = initialize_teacher_model(seeds[2], dropout_rates[2])


# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
student_model.to(device)
teacher_model1.to(device)
teacher_model2.to(device)
teacher_model3.to(device)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You sho

CustomTeacherModel(
  (model): XLMRobertaForSequenceClassification(
    (roberta): XLMRobertaModel(
      (embeddings): XLMRobertaEmbeddings(
        (word_embeddings): Embedding(250002, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): XLMRobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x XLMRobertaLayer(
            (attention): XLMRobertaAttention(
              (self): XLMRobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): XLMRobertaSelfOut

In [ ]:
len(dataset_train)

1842

In [ ]:
len(dataset_unlabeled)

6163

In [ ]:
# Set up optimizer and scheduler for the student model
optimizer = AdamW(student_model.parameters(), lr=1e-5, eps=1e-8)
epochs = 6
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataset_train) * epochs)

# Split labeled data into three equal parts with stratification for each teacher model
df_labeled_teacher1, df_temp_teacher = train_test_split(df_labeled, stratify=df_labeled[labels_column], test_size=2/3)
df_labeled_teacher2, df_labeled_teacher3 = train_test_split(df_temp_teacher, stratify=df_temp_teacher[labels_column], test_size=1/2)

# Function to tokenize data
def tokenize_data(df):
    encoded_data = tokenizer.batch_encode_plus(
        df[tweets_column].tolist(),
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        max_length=256,
        return_tensors='pt'
    )
    input_ids = encoded_data['input_ids']
    attention_masks = encoded_data['attention_mask']
    labels = torch.tensor(df[labels_column].values)
    return TensorDataset(input_ids, attention_masks, labels)

# Tokenize and create datasets for each teacher
dataset_teacher1 = tokenize_data(df_labeled_teacher1)
dataset_teacher2 = tokenize_data(df_labeled_teacher2)
dataset_teacher3 = tokenize_data(df_labeled_teacher3)

# Training loop with student and ensemble teachers
def calculate_alpha1(epoch, total_epochs, base_alpha=0.95, final_alpha=0.999):
    alpha = base_alpha + (final_alpha - base_alpha) * (epoch / total_epochs)
    return alpha

def calculate_alpha2(epoch, total_epochs, base_alpha=0.95, final_alpha=0.999):
    alpha = base_alpha + (final_alpha - base_alpha) * (epoch / total_epochs)
    return alpha

def calculate_alpha3(epoch, total_epochs, base_alpha=0.95, final_alpha=0.999):
    alpha = base_alpha + (final_alpha - base_alpha) * (epoch / total_epochs)
    return alpha

def update_teacher(teacher_model, student_model, alpha):
    with torch.no_grad():
        for teacher_param, student_param in zip(teacher_model.parameters(), student_model.parameters()):
            teacher_param.data.mul_(alpha).add_(student_param.data, alpha=(1 - alpha))

def compute_uncertainty(logits_teacher1, logits_teacher2, logits_teacher3):
    variance = torch.var(torch.stack([logits_teacher1, logits_teacher2, logits_teacher3]), dim=0)
    uncertainty = torch.mean(variance, dim=-1)
    return uncertainty

def compute_metrics(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    accuracy = accuracy_score(labels_flat, preds_flat)
    f1 = f1_score(labels_flat, preds_flat, average='weighted')
    precision = precision_score(labels_flat, preds_flat, average='weighted')
    recall = recall_score(labels_flat, preds_flat, average='weighted')
    return accuracy, f1, precision, recall


from tqdm import tqdm
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:

# Function to label pseudo-data using teacher models with confidence threshold
def label_pseudo_data(teacher_model1, teacher_model2, teacher_model3, dataset_unlabeled, confidence_threshold=0.95):
    pseudo_labels = []
    pseudo_confidence = []
    max_confidence = 0  # Initialize max confidence variable

    data_loader_unlabeled = DataLoader(dataset_unlabeled, sampler=SequentialSampler(dataset_unlabeled), batch_size=8)
    for batch in tqdm(data_loader_unlabeled, desc="Labeling Pseudo-Data"):
        batch = tuple(b.to(device) for b in batch)
        with torch.no_grad():
            logits1 = teacher_model1(input_ids=batch[0], attention_mask=batch[1]).logits
            logits2 = teacher_model2(input_ids=batch[0], attention_mask=batch[1]).logits
            logits3 = teacher_model3(input_ids=batch[0], attention_mask=batch[1]).logits
            ensemble_logits = (logits1 + logits2 + logits3) / 3
            probabilities = torch.nn.functional.softmax(ensemble_logits, dim=-1)
            confidences, predicted_labels = torch.max(probabilities, dim=-1)

        # Track the maximum confidence
        max_confidence = max(max_confidence, confidences.max().item())

        print(f'max_confidence: {max_confidence}')

        # Append all predicted labels and confidences to the lists
        pseudo_labels.append(predicted_labels)
        pseudo_confidence.append(confidences)

    # Concatenate all pseudo labels and confidences
    pseudo_labels = torch.cat(pseudo_labels, dim=0)
    pseudo_confidence = torch.cat(pseudo_confidence, dim=0)

    # After the loop in label_pseudo_data function
    print(f'Total pseudo-labels generated: {len(pseudo_labels[pseudo_confidence >= 0.95])}')


    return pseudo_labels, pseudo_confidence


In [ ]:
import itertools
from tqdm import tqdm
# here ran for 2 already
# Training loop with IPL and pseudo-labeled data integration
for epoch in range(1, epochs + 1):
    student_model.train()
    loss_train_total = 0

    # Create DataLoaders for the current epoch
    train_loader = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=8)
    unlabeled_loader = DataLoader(dataset_unlabeled, sampler=RandomSampler(dataset_unlabeled), batch_size=8)

    # Create iterators for training and unlabeled batches
    train_iterator = iter(train_loader)
    unlabeled_iterator = iter(unlabeled_loader)

    # Calculate number of batches
    num_train_batches = len(train_loader)
    num_unlabeled_batches = len(unlabeled_loader)
    max_batches = max(num_train_batches, num_unlabeled_batches)

    # Progress bar for the maximum number of batches
    progress_bar = tqdm(range(max_batches), desc=f'Epoch {epoch}', leave=False)

    for _ in progress_bar:
        try:
            # Get the next training batch
            train_batch = next(train_iterator)
        except StopIteration:
            train_iterator = iter(train_loader)  # Reset the iterator
            train_batch = next(train_iterator)

        train_batch = tuple(b.to(device) for b in train_batch)
        inputs = {'input_ids': train_batch[0], 'attention_mask': train_batch[1], 'labels': train_batch[2]}

        # Train the student on the labeled data (supervised loss)
        outputs_student = student_model(**inputs)
        loss_supervised = outputs_student.loss

        # Get the corresponding unlabeled batch
        try:
            unlabeled_batch = next(unlabeled_iterator)
        except StopIteration:
            unlabeled_iterator = iter(unlabeled_loader)  # Reset the iterator
            unlabeled_batch = next(unlabeled_iterator)

        unlabeled_batch = tuple(b.to(device) for b in unlabeled_batch)

        with torch.no_grad():
            logits_teacher1 = teacher_model1(input_ids=unlabeled_batch[0], attention_mask=unlabeled_batch[1]).logits
            logits_teacher2 = teacher_model2(input_ids=unlabeled_batch[0], attention_mask=unlabeled_batch[1]).logits
            logits_teacher3 = teacher_model3(input_ids=unlabeled_batch[0], attention_mask=unlabeled_batch[1]).logits
            ensemble_logits = (logits_teacher1 + logits_teacher2 + logits_teacher3) / 3

            # Compute uncertainty
            uncertainty = compute_uncertainty(logits_teacher1, logits_teacher2, logits_teacher3)
            uncertainty_weight = 1 / (1 + uncertainty)

        # Student model predictions on unlabeled data
        outputs_student_unlabeled = student_model(input_ids=unlabeled_batch[0], attention_mask=unlabeled_batch[1])
        loss_consistency = F.mse_loss(outputs_student_unlabeled.logits, ensemble_logits) * uncertainty_weight.mean()

        # Total loss
        loss = loss_supervised + loss_consistency
        loss_train_total += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(student_model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        # Cyclic teacher update based on the epoch number
        if epoch % 3 == 1:
            alpha1 = calculate_alpha1(epoch - (epoch / 3), (epochs * (2)) / 3)
            alpha2 = calculate_alpha2(epoch - (epoch / 3), (epochs * (2)) / 3)
            update_teacher(teacher_model1, student_model, alpha1)
            update_teacher(teacher_model2, student_model, alpha2)
        elif epoch % 3 == 2:
            alpha2 = calculate_alpha2(epoch - (epoch / 3), (epochs * (2)) / 3)
            alpha3 = calculate_alpha3(epoch - (epoch / 3), (epochs * (2)) / 3)
            update_teacher(teacher_model2, student_model, alpha2)
            update_teacher(teacher_model3, student_model, alpha3)
        elif epoch % 3 == 0:
            alpha1 = calculate_alpha1(epoch - (epoch / 3), (epochs * (2)) / 3)
            alpha3 = calculate_alpha3(epoch - (epoch / 3), (epochs * (2)) / 3)
            update_teacher(teacher_model3, student_model, alpha3)
            update_teacher(teacher_model1, student_model, alpha1)

    # Print average loss for the epoch
    print(f'\nEpoch {epoch}: Loss: {loss_train_total / (max_batches * 8):.3f}')  # Total loss normalized by total batches

    # Label pseudo-data using the teacher models after every epoch and append to labeled data
    if epoch % 1 == 0:
        pseudo_labels, pseudo_confidence = label_pseudo_data(teacher_model1, teacher_model2, teacher_model3, dataset_unlabeled, confidence_threshold=0.95)

        # Select only pseudo-labeled examples where confidence exceeds threshold
        if len(pseudo_labels) > 0:
            num_pseudo_labels = (pseudo_confidence >= 0.95).sum().item()
            input_ids_unlabeled = dataset_unlabeled.tensors[0]
            attention_masks_unlabeled = dataset_unlabeled.tensors[1]
            pseudo_confidence = pseudo_confidence.cpu()

            # Select pseudo-labeled data based on confidence threshold
            input_ids_pseudo = input_ids_unlabeled[pseudo_confidence >= 0.95]
            attention_masks_pseudo = attention_masks_unlabeled[pseudo_confidence >= 0.95]
            labels_pseudo = pseudo_labels[pseudo_confidence >= 0.95]

            # Remove pseudo-labeled data from the unlabeled dataset
            mask = pseudo_confidence < 0.95
            num_removed_from_unlabeled = (pseudo_confidence >= 0.95).sum().item()
            dataset_unlabeled = TensorDataset(input_ids_unlabeled[mask], attention_masks_unlabeled[mask])

            # Update the training dataset with pseudo-labeled data
            input_ids_combined = torch.cat((input_ids_train.to(device), input_ids_pseudo.to(device)), dim=0)
            attention_masks_combined = torch.cat((attention_masks_train.to(device), attention_masks_pseudo.to(device)), dim=0)
            labels_combined = torch.cat((labels_train.to(device), labels_pseudo.to(device)), dim=0)

            # Create a new training dataset with both real labels and pseudo-labels
            dataset_train = TensorDataset(input_ids_combined, attention_masks_combined, labels_combined)

            # Update input_ids_train and attention_masks_train for future iterations
            input_ids_train = input_ids_combined
            attention_masks_train = attention_masks_combined
            labels_train = labels_combined

            print(f'Pseudo-labeling completed for epoch {epoch}.')
            print(f'Number of pseudo-labels added: {num_pseudo_labels}')
            print(f'Combined training set now has {len(dataset_train)} examples.')
            print(f'Number of unlabeled samples removed: {num_removed_from_unlabeled}')
            print(f'Combined unlabeled set now has {len(dataset_unlabeled)} examples.')
        else:
            print(f'No pseudo-labels above the threshold for epoch {epoch}.')



      # Validation loop
    student_model.eval()
    teacher_model1.eval()
    teacher_model2.eval()
    teacher_model3.eval()
    loss_val_total = 0
    predictions_student, predictions_teacher1, predictions_teacher2, predictions_teacher3, true_vals = [], [], [], [], []

    progress_bar_val = tqdm(DataLoader(dataset_val, sampler=SequentialSampler(dataset_val), batch_size=8),
                            desc=f'Validation Epoch {epoch}', leave=False)

    for batch in progress_bar_val:
        batch = tuple(b.to(device) for b in batch)
        with torch.no_grad():
            # Student model predictions
            outputs_student = student_model(input_ids=batch[0], attention_mask=batch[1])
            logits_student = outputs_student.logits
            loss_val = F.cross_entropy(logits_student, batch[2])
            loss_val_total += loss_val.item()

            # Teacher model predictions
            logits_teacher1 = teacher_model1(input_ids=batch[0], attention_mask=batch[1]).logits
            logits_teacher2 = teacher_model2(input_ids=batch[0], attention_mask=batch[1]).logits
            logits_teacher3 = teacher_model3(input_ids=batch[0], attention_mask=batch[1]).logits

            # Collect predictions and true labels
            predictions_student.append(logits_student.detach().cpu().numpy())
            predictions_teacher1.append(logits_teacher1.detach().cpu().numpy())
            predictions_teacher2.append(logits_teacher2.detach().cpu().numpy())
            predictions_teacher3.append(logits_teacher3.detach().cpu().numpy())
            true_vals.append(batch[2].cpu().numpy())

        progress_bar_val.set_postfix({'validation_loss': f'{loss_val.item():.3f}'})

    # Concatenate predictions and true labels
    predictions_student = np.concatenate(predictions_student, axis=0)
    predictions_teacher1 = np.concatenate(predictions_teacher1, axis=0)
    predictions_teacher2 = np.concatenate(predictions_teacher2, axis=0)
    predictions_teacher3 = np.concatenate(predictions_teacher3, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    # Compute metrics for student and teacher models
    metrics_student = compute_metrics(predictions_student, true_vals)
    metrics_teacher1 = compute_metrics(predictions_teacher1, true_vals)
    metrics_teacher2 = compute_metrics(predictions_teacher2, true_vals)
    metrics_teacher3 = compute_metrics(predictions_teacher3, true_vals)

    # Print validation metrics
    print(f'Epoch {epoch}: Validation Loss: {loss_val_total / len(dataset_test):.3f}')
    print(f'Student Model - Accuracy: {metrics_student[0]:.3f}, F1 Score: {metrics_student[1]:.3f}, Precision: {metrics_student[2]:.3f}, Recall: {metrics_student[3]:.3f}')
    print(f'Teacher Model 1 - Accuracy: {metrics_teacher1[0]:.3f}, F1 Score: {metrics_teacher1[1]:.3f}, Precision: {metrics_teacher1[2]:.3f}, Recall: {metrics_teacher1[3]:.3f}')
    print(f'Teacher Model 2 - Accuracy: {metrics_teacher2[0]:.3f}, F1 Score: {metrics_teacher2[1]:.3f}, Precision: {metrics_teacher2[2]:.3f}, Recall: {metrics_teacher2[3]:.3f}')
    print(f'Teacher Model 3 - Accuracy: {metrics_teacher3[0]:.3f}, F1 Score: {metrics_teacher3[1]:.3f}, Precision: {metrics_teacher3[2]:.3f}, Recall: {metrics_teacher3[3]:.3f}')


Epoch 1: Loss: 0.062


Labeling Pseudo-Data:   0%|          | 1/771 [00:00<04:53,  2.62it/s]

max_confidence: 0.779670000076294


Labeling Pseudo-Data:   0%|          | 2/771 [00:00<04:30,  2.84it/s]

max_confidence: 0.7829613089561462


...

Labeling Pseudo-Data: 100%|██████████| 771/771 [04:20<00:00,  2.96it/s]

max_confidence: 0.9768514037132263
max_confidence: 0.9768514037132263
Total pseudo-labels generated: 4463


Pseudo-labeling completed for epoch 3.
Number of pseudo-labels added: 4463
Combined training set now has 6305 examples.
Number of unlabeled samples removed: 4463
Combined unlabeled set now has 1700 examples.


Epoch 3: Validation Loss: 0.048
Student Model - Accuracy: 0.867, F1 Score: 0.866, Precision: 0.872, Recall: 0.867
Teacher Model 1 - Accuracy: 0.856, F1 Score: 0.855, Precision: 0.868, Recall: 0.856
Teacher Model 2 - Accuracy: 0.862, F1 Score: 0.861, Precision: 0.876, Recall: 0.862
Teacher Model 3 - Accuracy: 0.856, F1 Score: 0.855, Precision: 0.868, Recall: 0.856


Epoch 4:   6%|▌         | 49/789 [00:56<14:12,  1.15s/it]

In [ ]:
# Testing loop
student_model.eval()
teacher_model1.eval()
teacher_model2.eval()
teacher_model3.eval()
loss_test_total = 0
predictions_student, predictions_ensemble, true_vals = [], [], []

# Use tqdm for the testing loop
progress_bar_test = tqdm(DataLoader(dataset_test, sampler=SequentialSampler(dataset_test), batch_size=8),
                         desc='Testing',
                         leave=False,
                         disable=False)

for batch in progress_bar_test:
    batch = tuple(b.to(device) for b in batch)
    with torch.no_grad():
        # Student model predictions
        outputs_student = student_model(input_ids=batch[0], attention_mask=batch[1])
        logits_student = outputs_student.logits
        loss_test = F.cross_entropy(logits_student, batch[2])
        loss_test_total += loss_test.item()

        # Teacher model predictions
        logits_teacher1 = teacher_model1(input_ids=batch[0], attention_mask=batch[1]).logits
        logits_teacher2 = teacher_model2(input_ids=batch[0], attention_mask=batch[1]).logits
        logits_teacher3 = teacher_model3(input_ids=batch[0], attention_mask=batch[1]).logits

        # Ensemble predictions (average of teacher logits)
        logits_ensemble = (logits_teacher1 + logits_teacher2 + logits_teacher3) / 3

        # Collect predictions and true labels
        predictions_student.append(logits_student.detach().cpu().numpy())
        predictions_ensemble.append(logits_ensemble.detach().cpu().numpy())
        true_vals.append(batch[2].cpu().numpy())

    # Update the progress bar with the current test loss
    progress_bar_test.set_postfix({'test_loss': f'{loss_test.item():.3f}'})

# Concatenate predictions and true labels
predictions_student = np.concatenate(predictions_student, axis=0)
predictions_ensemble = np.concatenate(predictions_ensemble, axis=0)
true_vals = np.concatenate(true_vals, axis=0)

# Compute metrics for student model
accuracy_student, f1_student, precision_student, recall_student = compute_metrics(predictions_student, true_vals)

# Compute metrics for ensemble of teacher models
accuracy_ensemble, f1_ensemble, precision_ensemble, recall_ensemble = compute_metrics(predictions_ensemble, true_vals)

# Print testing metrics for student and ensembled teacher models
print(f'Test Loss: {loss_test_total / len(dataset_test):.3f}')
print(f'Student Model - Accuracy: {accuracy_student:.3f}, F1 Score: {f1_student:.3f}, Precision: {precision_student:.3f}, Recall: {recall_student:.3f}')
print(f'Ensembled Teacher Models - Accuracy: {accuracy_ensemble:.3f}, F1 Score: {f1_ensemble:.3f}, Precision: {precision_ensemble:.3f}, Recall: {recall_ensemble:.3f}')



NameError: name 'student_model' is not defined

In [ ]:
# ---- LIME explainability with average of both student models ----

from lime.lime_text import LimeTextExplainer  # LIME Import
# LIME for explainability: explain the output using the average of both student models
class_names = list(label_dict.keys())
explainer = LimeTextExplainer(class_names=class_names, split_expression='\s+')

def predict_average_students(texts):
    encodings = tokenizer.batch_encode_plus(texts, add_special_tokens=True, return_attention_mask=True, pad_to_max_length=True, max_length=256, return_tensors='pt')
    input_ids = encodings['input_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)

    with torch.no_grad():
        # Forward pass through both student models
        outputs_student1 = teacher_model1(input_ids=input_ids, attention_mask=attention_mask)
        outputs_student2 = teacher_model2(input_ids=input_ids, attention_mask=attention_mask)
        outputs_student3 = teacher_model3(input_ids=input_ids, attention_mask=attention_mask)

        # Average the logits of both student models
        avg_logits = (outputs_student1.logits + outputs_student2.logits+outputs_student3.logits ) / 3.0

        # Convert logits to probabilities
        probs = F.softmax(avg_logits, dim=1).detach().cpu().numpy()

    return probs

# Explain a random sample from the test set
random_idx = random.randint(0, len(df_test) - 1)
text_sample = df_test.iloc[random_idx][tweets_column]
true_label = df_test.iloc[random_idx][labels_column]
print(f"Sample text: {text_sample}")
print(f"True label: {class_names[true_label]}")

# Use the new prediction function for LIME
exp = explainer.explain_instance(text_sample, predict_average_students, num_features=10,num_samples=100)
exp.show_in_notebook()